Import of necessary libraries and dataset read

In [ ]:
import pandas as pd


dataset= pd.read_csv("explaination_toxic_conversation_most_toxic_sentences.csv",header=0, index_col=0, encoding='utf-8')
conversation, explaination = dataset.iloc[0][['conversation', 'explaination']]
print("Conversation:", conversation)
print("Explaination:", explaination)


Conversation: Liviero: "Spero che tu sia felice di essere rientrata dal lavoro. Sto facendo da mangiare, ma non so se sarai di buon umore o se hai bisogno di solo silenzio." Vannina: "Non ho bisogno di silenzio. Non ho bisogno di niente. Solo di essere lasciata in pace." Liviero: "Non capisco. Cosa ho fatto di male? Perché sei così arrabbiata con me?" Vannina: "Non sei arrabbiata, sei solo insopportabile. Non puoi fare il tuo lavoro e il mio senza stressarmi." Liviero: "Ma non ho fatto nulla di male. Sto solo cercando di aiutarti." Vannina: "Tu non aiuti. Aiuti solo a peggiorare le cose. Non vedi che sono felice solo quando sei lontano?"
Explaination: La conversazione è tossica perché Liviero cerca di controllare Vannina, imponendole il suo desiderio di silenzio e di essere accudita. Vannina, però, si rifiuta di essere manipolata e si sente stressata e oppressa dal comportamento di Liviero. Questo tipo di conversazione è caratterizzato dalla mancanza di empatia e dalla tendenza a giust